In [5]:
from fobokiller.heatmap import load_reviews_dataset, heatmap_sentences, \
load_model,apply_heatmap_html,apply_heatmap_polarity

from fastapi import FastAPI, Query
from fastapi.middleware.cors import CORSMiddleware
import pandas as pd
from typing import List

from fobokiller.cosim import compute_sim_df, load_embedding, summary_reviews
#words/sentences  preprocessing
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import backend as K
#modeling
from tensorflow import GradientTape
import tensorflow as tf#modeling
from tensorflow.keras import models, layers

In [6]:
resto_list = pd.read_csv("../api/final_resto_list.csv", index_col=0)

In [7]:




embedding = load_embedding()

reviews_dataset = load_reviews_dataset()

model_heatmap = load_model()


In [8]:


app = FastAPI()


app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Allows all origins
    allow_credentials=True,
    allow_methods=["*"],  # Allows all methods
    allow_headers=["*"],  # Allows all headers
)


temp =[]
result = []

@app.get("/")
def index():
    return {"greeting": "Hello world"}


#Input list of restaurant (alias)
# detail
@app.get("/detail")
def get_details(alias):

    alias = [alias]
    pd_liste = resto_list.loc[resto_list['alias'].isin(alias), :]
    return pd_liste.to_dict()



@app.get("/details/")
def read_items(alias: List[str] = Query(None)):
    print(alias)
    pd_liste = resto_list.loc[resto_list['alias'].isin(alias), :]
    print(pd_liste)

    return pd_liste.to_dict()

@app.get("/summary_reviews")
def sr(text, n_best=10, n_prox=3000, min_review=0):
    min_review = int(min_review)
    print(type(n_prox))
    if pd.isna(n_prox):
        pass
    else:

        n_prox = int(n_prox)
    temp = compute_sim_df(text, embedding, n_prox, min_review)
    n_best = int(n_best)
    result = summary_reviews(temp, n_best)
    return result.to_dict()



@app.get("/summary_reviews2")
def sr2(text, n_best=1, n_prox=3000, min_review=10):

    #setting types
    min_review = int(min_review)
    if pd.isna(n_prox):
        pass
    else:
        n_prox = int(n_prox)
    n_best = int(n_best)

    #loading datasets
    results = compute_sim_df(text, embedding, n_prox, min_review)
    summary = summary_reviews(results, n_best)
    results['is_sim'] = 1
    summary['is_in_summary'] = 1
    results_trimed = results.drop(
        columns=['alias', 'rate', 'review_sentences'])

    #merging datasets and house cleaning

    tmp_df = reviews_dataset.merge(results_trimed,
                                   on='review_clean',
                                   how='left')
    tmp_df['is_sim'].fillna(0, inplace=True)
    all_df = tmp_df.merge(summary, on='alias', how='left')
    all_df.fillna(0, inplace=True)
    all_df = all_df[all_df["is_in_summary"]==1]

    #apply heatmap for html and polarity score
    all_df['reviews_heatmaps_html'] = all_df.apply(apply_heatmap_html,axis=1)
    all_df['reviews_heatmaps_polarity'] = all_df.apply(apply_heatmap_polarity, axis=1)
    ####  metrics for the val of the request
    all_df['request_metric'] = all_df[(all_df['is_in_summary'] == 1) & (
        all_df['is_sim'] == 1)]['nb_sentences'].sum() *100 /3000
    summary_reconstructed = all_df.groupby('alias').agg({
        'review_clean':
        list,
        'nb_sentences':
        'mean',
        'nb_review':
        'mean',
        'metric sim_ratio':
        'mean',
        'reviews_heatmaps_html':
        list,
        'reviews_heatmaps_polarity':
        list
    })

    #print("check")
    #print(type(summary_reconstructed))
    #print(summary_reconstructed.columns)
    #print(summary_reconstructed.shape)
    output_json = summary_reconstructed.to_dict()
    return output_json


In [27]:
%%time
text = "I want a burger"
n_best=10
n_prox=3000
min_review=10


min_review = int(min_review)
if pd.isna(n_prox):
    pass
else:
    n_prox = int(n_prox)
n_best = int(n_best)

CPU times: user 106 µs, sys: 2.09 ms, total: 2.2 ms
Wall time: 2.5 ms


In [28]:
%%time
#loading datasets
results = compute_sim_df(text, embedding, n_prox, min_review)
summary = summary_reviews(results, n_best)
results['is_sim'] = 1
summary['is_in_summary'] = 1
results_trimed = results.drop(
        columns=['alias', 'rate', 'review_sentences'])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [35]:
%%time
#merging datasets and house cleaning

tmp_df = reviews_dataset.merge(results_trimed,
                                   on='review_clean',
                                   how='left')
tmp_df['is_sim'].fillna(0, inplace=True)
all_df = tmp_df.merge(summary, on='alias', how='left')
all_df.fillna(0, inplace=True)
all_df = all_df[all_df["is_in_summary"]==1]


CPU times: user 207 ms, sys: 804 ms, total: 1.01 s
Wall time: 2.17 s


In [51]:
%%time
#apply heatmap for html and polarity score
all_df['reviews_heatmaps_html'] = all_df.apply(apply_heatmap_html,axis=1)


CPU times: user 31.1 s, sys: 2.31 s, total: 33.5 s
Wall time: 31.4 s


In [56]:
from dask.multiprocessing import get
import dask.dataframe as dd
ddata = dd.from_pandas(all_df, npartitions=30)

In [58]:
es = ddata.map_partitions(lambda df: df.apply((lambda row: count(*row)), axis=1)).compute(get=get) 

ValueError: Metadata inference failed in `lambda`.

You have supplied a custom function and Dask is unable to 
determine the type of output that that function returns. 

To resolve this please provide a meta= keyword.
The docstring of the Dask function you ran should have more information.

Original error is below:
------------------------
NameError("name 'count' is not defined")

Traceback:
---------
  File "/Users/manoharan/.pyenv/versions/3.8.12/envs/FOBO/lib/python3.8/site-packages/dask/dataframe/utils.py", line 176, in raise_on_meta_error
    yield
  File "/Users/manoharan/.pyenv/versions/3.8.12/envs/FOBO/lib/python3.8/site-packages/dask/dataframe/core.py", line 5833, in _emulate
    return func(*_extract_meta(args, True), **_extract_meta(kwargs, True))
  File "/var/folders/s6/nxj_k6112bj7cnd89_8b9l0c0000gn/T/ipykernel_72677/617588866.py", line 1, in <lambda>
    es = ddata.map_partitions(lambda df: df.apply((lambda row: count(*row)), axis=1)).compute(get=get)
  File "/Users/manoh

In [39]:
[i for i in all_df]

['review_clean',
 'alias',
 'rate',
 'review_sentences',
 'embedding',
 'review_sentences_trimed',
 'date',
 'sim_s',
 'rate_filtered_x',
 'review_filtered',
 'sim_r',
 'rate_all',
 'review_all',
 'ratio',
 'metric',
 'is_sim',
 'reviews',
 'nb_sentences',
 'nb_review',
 'metric sim_ratio',
 'rate_filtered_y',
 'is_in_summary',
 'reviews_heatmaps_html']

In [38]:
Parallel(n_jobs=-1)(delayed(apply_heatmap_html))

TypeError: __call__() got an unexpected keyword argument 'axis'

In [26]:
%%time
all_df['reviews_heatmaps_html'] = all_df[:20].apply(apply_heatmap_html,axis=1)

CPU times: user 1.78 s, sys: 237 ms, total: 2.01 s
Wall time: 2.06 s


In [31]:
%%time
all_df['reviews_heatmaps_polarity'] = all_df.apply(apply_heatmap_polarity, axis=1)

CPU times: user 35.7 s, sys: 1.96 s, total: 37.7 s
Wall time: 43.1 s


In [34]:
all_df

,review_clean,alias,rate,review_sentences,embedding,review_sentences_trimed,date,sim_s,rate_filtered_x,review_filtered,...,ratio,metric,is_sim,reviews,nb_sentences,nb_review,metric sim_ratio,rate_filtered_y,is_in_summary,reviews_heatmaps_polarity
28,"""i still eat a burger at a counter with ketchu...",blend-paris-26,4.0,"[""i still eat a burger at a counter with ketch...","[[0.04524126, 0.093474865, 0.018982986, -0.091...","[""i still eat a burger at a counter with ketch...",2021-01-26,0.689196,3.928571,61.0,...,0.753086,0.279523,1.0,{after visiting the louvre museum we looked fo...,98.0,61.0,0.279523,3.928571,1.0,0.979778
29,"""i still eat a burger at a counter with ketchu...",blend-paris-26,4.0,"[""i still eat a burger at a counter with ketch...","[[0.04524126, 0.093474865, 0.018982986, -0.091...","[""i still eat a burger at a counter with ketch...",2021-01-26,0.625217,3.928571,61.0,...,0.753086,0.279523,1.0,{after visiting the louvre museum we looked fo...,98.0,61.0,0.279523,3.928571,1.0,0.979778
30,"""i still eat a burger at a counter with ketchu...",blend-paris-26,4.0,"[""i still eat a burger at a counter with ketch...","[[0.04524126, 0.093474865, 0.018982986, -0.091...","[""i still eat a burger at a counter with ketch...",2021-01-26,0.440658,3.928571,61.0,...,0.753086,0.279523,1.0,{after visiting the louvre museum we looked fo...,98.0,61.0,0.279523,3.928571,1.0,0.979778
285,"a ""french"" take on an amazing burger joint but...",blend-paris-26,4.0,"[a ""french"" take on an amazing burger joint bu...","[[0.038462874, -0.015068263, 0.021961713, -0.0...","[a ""french"" take on an amazing burger joint bu...",2012-03-04,0.409594,3.928571,61.0,...,0.753086,0.279523,1.0,{after visiting the louvre museum we looked fo...,98.0,61.0,0.279523,3.928571,1.0,0.974979
404,a cool little (and i mean little) gourmet burg...,blend-paris-26,4.0,[a cool little (and i mean little) gourmet bur...,"[[0.049561992, 0.009552067, 0.010170949, -0.01...",[a cool little (and i mean little) gourmet bur...,2015-03-02,0.520061,3.928571,61.0,...,0.753086,0.279523,1.0,{after visiting the louvre museum we looked fo...,98.0,61.0,0.279523,3.928571,1.0,0.975982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56340,wow! great burger. in the quest to find the be...,big-fernand-paris-12,4.0,"[wow, great burger, in the quest to find the...","[[-0.02499111, 0.07090064, -0.033703253, 0.040...","[wow, great burger, in the quest to find the...",2013-03-05,0.564804,4.162162,21.0,...,0.677419,0.257047,1.0,{it's a crazy burger place! line was out the d...,37.0,21.0,0.257047,4.162162,1.0,0.978563
56350,wow! if you really like burgers and you want y...,little-cantine-paris,5.0,"[wow, if you really like burgers and you want...","[[-0.02499111, 0.07090064, -0.033703253, 0.040...","[wow, if you really like burgers and you want...",2018-08-15,0.554000,4.675676,27.0,...,0.586957,0.254946,1.0,{amazing burger! get the dude burger -- cheese...,37.0,27.0,0.254946,4.675676,1.0,0.966912
56351,wow! if you really like burgers and you want y...,little-cantine-paris,5.0,"[wow, if you really like burgers and you want...","[[-0.02499111, 0.07090064, -0.033703253, 0.040...","[wow, if you really like burgers and you want...",2018-08-15,0.413223,4.675676,27.0,...,0.586957,0.254946,1.0,{amazing burger! get the dude burger -- cheese...,37.0,27.0,0.254946,4.675676,1.0,0.966912
57000,young and hip crowd..went around : and there w...,blend-paris-26,3.0,"[young and hip crowd, went around : and there ...","[[-0.018133149, -0.011460958, 0.014708627, -0....","[young and hip crowd, went around : and there ...",0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,{after visiting the louvre museum we looked fo...,98.0,61.0,0.279523,3.928571,1.0,0.000000


In [ ]:
similarities_parra = Parallel(n_jobs=-1)(
        delayed(util.cos_sim)(input_encoded, split_embedding)
        for split_embedding in embedding_split)

In [32]:
from joblib import Parallel, delayed
import multiprocessing
import torch

In [ ]:
Parallel(n_jobs=-1)(delayed(apply_heatmap_polarity))

In [22]:
%%time
####  metrics for the val of the request
all_df['request_metric'] = all_df[(all_df['is_in_summary'] == 1) & (
        all_df['is_sim'] == 1)]['nb_sentences'].sum() *100 /3000
summary_reconstructed = all_df.groupby('alias').agg({
        'review_clean':
        list,
        'nb_sentences':
        'mean',
        'nb_review':
        'mean',
        'metric sim_ratio':
        'mean',
        'reviews_heatmaps_html':
        list,
        'reviews_heatmaps_polarity':
        list
    })


CPU times: user 11.5 ms, sys: 4.04 ms, total: 15.6 ms
Wall time: 20.6 ms


In [23]:
%%time
    #print("check")
    #print(type(summary_reconstructed))
    #print(summary_reconstructed.columns)
    #print(summary_reconstructed.shape)
output_json = summary_reconstructed.to_dict()
 


CPU times: user 1.11 ms, sys: 119 µs, total: 1.23 ms
Wall time: 1.21 ms


In [24]:
output_json

{'review_clean': {'big-fernand-paris-12': ['an exceptionally common complaint of american expats in europe is that you can\'t find good mexican food (understandable) and you can\'t find good burgers (less understandable, but equally true). well fear not, because while the burgers at big fernand might not necessarily displace your favorite burger from back in the states, it will most certainly check the box if you\'re having a craving. star : how were the service and atmosphere? other reviews are telling me that the line here at big fernand can grow quite lengthy, but there were only five or six others when i arrived fifteen minutes prior to opening on a saturday evening. the line moves quite quickly and there\'s a good quantity of seating, and all this comes with an efficient and friendly staff who not only speak english but are more than happy to give you a fully english menu. after ordering, i was absolutely amazed at how fast my burger came out (under a minute maybe? it was amazing)